# Surface Poisson equation

In this notebook we will solve a Poisson equation on a non-closed surface with homogeneous and inhomogeneous Dirichlet boundary conditions.

## Homogeneous Dirichlet data

In [ ]:
from ngsolve import *
from netgen.occ import *
from ngsolve.webgui import Draw

Given a half-sphere we want to solve the surface Poisson equation $$\int_S\nabla_{\Gamma}u\cdot\nabla_{\Gamma}v\,ds = \int_S fv\,ds,\qquad u=0 \text{ on } \Gamma_D.$$

Generate geometry and curved surface mesh.

In [ ]:
# sphere and halfspace
sphere = Sphere((0, 0, 0), 1)
sphere.faces.name = "sphere"
plane = HalfSpace((0, 0, 0), (0, 0, -1))
# intersect to get halfsphere and take faces
halfsphere = (sphere * plane).faces["sphere"][0]
# name edges
halfsphere.edges[Z <= 0].name = "bottom"
mesh = Mesh(OCCGeometry(halfsphere).GenerateMesh(maxh=0.3))
mesh.Curve(2)
Draw(mesh);

We define the usual $H^1$ finite element space and use the <i>dirichlet_bbnd</i> flag to indicate the BBoundary (Co-dimension 2) on which the Dirichlet conditions are prescribed. The test- and trial-functions are given as usual.

In [ ]:
fes = H1(mesh, order=2, dirichlet_bbnd="bottom")
u, v = fes.TnT()  # Trial and Test
print(fes.FreeDofs())

For the (bi-)linear form we have to take care that we have to define the according integrators on the boundary and that the <i>Trace</i> operator has to be used to obtain the tangential/surface derivative.

In [ ]:
a = BilinearForm(fes, symmetric=True)
a += grad(u).Trace() * grad(v).Trace() * ds
a.Assemble()

force = 20 * sin(x) * y * exp(z)

f = LinearForm(fes)
# Trace() for H1 functions not necessary
f += force * v.Trace() * ds
f.Assemble();

Solving is done as usual:

In [ ]:
gfu = GridFunction(fes)
gfu.vec.data = a.mat.Inverse(fes.FreeDofs()) * f.vec
Draw(gfu, mesh, "u")
Draw(Grad(gfu), mesh, "gradu");

## Inhomogeneous Dirichlet data

To solve the same problem with non-homogenous Dirichlet data, $u=u_D$ on $\Gamma_D$ the same technique as in the volume case is used, where we have to set a function on the BBoundary instead on the boundary.

In [ ]:
gfu.Set(x, definedon=mesh.BBoundaries("bottom"))
Draw(gfu)

r = f.vec.CreateVector()
r.data = f.vec - a.mat * gfu.vec
gfu.vec.data += a.mat.Inverse(fes.FreeDofs()) * r

Draw(gfu);

## Finite element spaces for surfaces


We distinguish between two types of finite element spaces for surfaces. The first class consists of spaces, where the restriction of a 3D element to the surface leads to a valid 2D element of the same type: H1, HCurl, HCurlCurl, NumberSpace.

These spaces can directly be used, one has to take care using the Trace operator. Otherwise an exception is thrown during assembling. 

In [ ]:
a += grad(u) * grad(v) * ds
a.Assemble();

The NumberSpace is an exception as it represents only a number, where no Trace operator has to be used.

The second class is given by

| Space     |  Surface Space |
|:----------|:---------------|
| L2 	       | SurfaceL2
| HDiv         | HDivSurface
| HDivDiv 	   | HDivDivSurface
| FacetFESpace | FacetSurface

Here, a 2D reference element is mapped directly onto the surface. To be consistent, also here the Trace operator has to be used.